# Model Unrolling 1

This is the first of several notebooks to explore unrolling in SysML v2 models to reach the point where a derived model has only a single way to be understood (all multiplicities constrained to 1, all features constrained to 1, all sets defined as unions of single-element sets).

## Pre-Requisites

This section of the code builds upon model loading and the Pythonic representation of a model, as well as the creation and modification of model elements.

Load the model from JSON generated by the KerML2JSON utility in the SysML Reference Implementation. The -d and -g flags are used to apply derived and implicit relationships before the export.

In [1]:
import json
with open('../data_loading/example_data/Clean_Bicycle.json', 'r') as fp:
    element_raw_data = json.load(fp)
    
printing_level = "TRACE"

In [2]:
import pymbe.api as pm
extracted_elements = [(lambda d: d.update({'@id': ele['identity']['@id']}) or d)(ele['payload'])
                      for ele in element_raw_data]
bicycle_model = pm.Model.load(extracted_elements)
bicycle_model

<SysML v2 Model (578 elements)>

## Developing unrolling rule:
### Find Features with multiplicity of 1, typed by non-1 multiplicity types.

Look for multiplicity ranges in the model that are set to 1.

In [24]:
def isMultiplicityOne(multiplicity_range):
    literal_value = [li.value for li in multiplicity_range.ownedElement if li['@type'] == 'LiteralInteger']
    if len(literal_value) == 0:
        return False
    elif len(literal_value) == 1:
        return literal_value[0] == 1
    elif len(literal_value) == 2:
        return literal_value[0] == 1 and literal_value[1] == 1

In [32]:
def isMultiplicitySpecificFinite(multiplicity_range):
    literal_value = [li.value for li in multiplicity_range.ownedElement if li['@type'] == 'LiteralInteger']
    if len(literal_value) == 0:
        return False
    elif len(literal_value) == 1:
        return literal_value[0] > 1
    elif len(literal_value) == 2:
        return literal_value[0] > 1 and literal_value[0] == literal_value[1]

In [39]:
def isTypeUndefinedMult(type_ele):
    if hasattr(element, "ownedElement") is False:
        return False
    mult_range = [mr for mr in element.ownedElement if mr['@type'] == 'MultiplicityRange']
    return len(mult_range) == 0

In [36]:
one_multiplicities = [ele for (ele_id, ele)
                      in bicycle_model.all_non_relationships.items()
                      if ele['@type'] == 'MultiplicityRange' and isMultiplicityOne(ele)]
print (f"Found {len(one_multiplicities)} single value multiplicities")

Found 33 single value multiplicities


In [38]:
finite_multiplicities = [ele for (ele_id, ele)
                      in bicycle_model.all_non_relationships.items()
                      if ele['@type'] == 'MultiplicityRange' and isMultiplicitySpecificFinite(ele)]
print (f"Found {len(finite_multiplicities)} finite value multiplicities")

Found 0 finite value multiplicities


Create a function to discover multiplicities on elements.

In [3]:
def navigateToMultiplicity(element):
    if printing_level == "TRACE":
        print(f"Trying to find multiplicity on type {element}")
    if hasattr(element, 'ownedElement'):
        mult_range = [mr for mr in element.ownedElement if mr['@type'] == 'MultiplicityRange']
        if len(mult_range) > 0:
            literal_value = [li.value for li in mult_range[0].ownedElement if li['@type'] == 'LiteralInteger']
            print(f"Found literal value on type {element}")
            return literal_value
        print(f"Found non-literal multiplicity range on type {element}")
        return mult_range
    if printing_level == "TRACE":
        print(f"Found no multiplicity on type {element}")
    return []

In [4]:
all_class_defs = [ele for (ele_id, ele)
                   in bicycle_model.all_non_relationships.items()
                   if ele['@type'] == 'Classifier']
all_class_defs

[<Anything «Classifier»>,
 <A Bike «Classifier»>,
 <A Seat «Classifier»>,
 <Wheel 1 «Classifier»>,
 <Wheel 2 «Classifier»>,
 <A Bike «Classifier»>,
 <A Seat «Classifier»>,
 <Wheel «Classifier»>,
 <A Bike «Classifier»>,
 <A Seat «Classifier»>,
 <Wheel 2 «Classifier»>,
 <Wheel 1 «Classifier»>,
 <Wheel «Classifier»>]

In [5]:
test_class = all_class_defs[2]

In [6]:
[(each_class, navigateToMultiplicity(each_class)) for each_class in all_class_defs]

Trying to find multiplicity on type <Anything «Classifier»>
Found no multiplicity on type <Anything «Classifier»>
Trying to find multiplicity on type <A Bike «Classifier»>
Found literal value on type <A Bike «Classifier»>
Trying to find multiplicity on type <A Seat «Classifier»>
Found literal value on type <A Seat «Classifier»>
Trying to find multiplicity on type <Wheel 1 «Classifier»>
Found literal value on type <Wheel 1 «Classifier»>
Trying to find multiplicity on type <Wheel 2 «Classifier»>
Found literal value on type <Wheel 2 «Classifier»>
Trying to find multiplicity on type <A Bike «Classifier»>
Found literal value on type <A Bike «Classifier»>
Trying to find multiplicity on type <A Seat «Classifier»>
Found literal value on type <A Seat «Classifier»>
Trying to find multiplicity on type <Wheel «Classifier»>
Found non-literal multiplicity range on type <Wheel «Classifier»>
Trying to find multiplicity on type <A Bike «Classifier»>
Found literal value on type <A Bike «Classifier»>
Try

[(<Anything «Classifier»>, []),
 (<A Bike «Classifier»>, [1]),
 (<A Seat «Classifier»>, [1]),
 (<Wheel 1 «Classifier»>, [1]),
 (<Wheel 2 «Classifier»>, [1]),
 (<A Bike «Classifier»>, [1]),
 (<A Seat «Classifier»>, [1]),
 (<Wheel «Classifier»>, []),
 (<A Bike «Classifier»>, [1]),
 (<A Seat «Classifier»>, [1]),
 (<Wheel 2 «Classifier»>, [1]),
 (<Wheel 1 «Classifier»>, [1]),
 (<Wheel «Classifier»>, [])]

In [7]:
all_feature_defs = [ele for (ele_id, ele)
                   in bicycle_model.all_non_relationships.items()
                   if ele['@type'] == 'Feature']

In [8]:
model_package = "'Example Bicycle Open Wheels'"

In [9]:
model_features = [(each_feature, 
  navigateToMultiplicity(each_feature)) for each_feature in all_feature_defs
 if each_feature.qualifiedName.split('::')[0] == model_package]
model_features

Trying to find multiplicity on type <7d8b376a-0644-4eef-a60e-2cbd2521abfb «Feature»>
Found non-literal multiplicity range on type <7d8b376a-0644-4eef-a60e-2cbd2521abfb «Feature»>
Trying to find multiplicity on type <e2132391-ab40-48ce-8339-f188f4ffca5c «Feature»>
Found non-literal multiplicity range on type <e2132391-ab40-48ce-8339-f188f4ffca5c «Feature»>
Trying to find multiplicity on type <aec7214d-a245-4da2-a5eb-66d1487afbc7 «Feature»>
Found non-literal multiplicity range on type <aec7214d-a245-4da2-a5eb-66d1487afbc7 «Feature»>
Trying to find multiplicity on type <d8815a76-4d96-4b27-80fc-9ec1e513ff2f «Feature»>
Found non-literal multiplicity range on type <d8815a76-4d96-4b27-80fc-9ec1e513ff2f «Feature»>
Trying to find multiplicity on type <My Seat «Feature»>
Found literal value on type <My Seat «Feature»>
Trying to find multiplicity on type <74bcdb7a-1ad9-4124-91af-05e9d02c6bc8 «Feature»>
Found non-literal multiplicity range on type <74bcdb7a-1ad9-4124-91af-05e9d02c6bc8 «Feature»>
T

[(<7d8b376a-0644-4eef-a60e-2cbd2521abfb «Feature»>, []),
 (<e2132391-ab40-48ce-8339-f188f4ffca5c «Feature»>, []),
 (<aec7214d-a245-4da2-a5eb-66d1487afbc7 «Feature»>, []),
 (<d8815a76-4d96-4b27-80fc-9ec1e513ff2f «Feature»>, []),
 (<My Seat «Feature»>, [1]),
 (<74bcdb7a-1ad9-4124-91af-05e9d02c6bc8 «Feature»>, []),
 (<Back Wheel «Feature»>, [1]),
 (<ddfff54f-3d57-4fcb-bbd4-562dd4140044 «Feature»>, []),
 (<Front Wheel «Feature»>, [1]),
 (<e9e42ab2-6b69-4cf5-81a8-6eba244b1d75 «Feature»>, []),
 (<driver «Feature»>, []),
 (<driven «Feature»>, []),
 (<8d6e35fb-539d-4adf-9a0f-3a651987eda8 «Feature»>, []),
 (<ce5aaff2-7868-4b89-8fb3-78b4b08a8f60 «Feature»>, []),
 (<30d92b09-5cdb-4d7f-8c9b-fb3f2b178ee5 «Feature»>, []),
 (<My Bike «Feature»>, [1])]

The following are functions to support the rules described in canonical instantiation.

In [10]:
def getFiniteMultiplicityTypes(elements_list, package):
    model_types = [each_type for each_type in elements_list
                      if each_type.qualifiedName.split('::')[0] == package
                     ]
    if printing_level == "TRACE":
        print(model_types)
    return [finite_type for finite_type in model_types if
            len(navigateToMultiplicity(finite_type)) == 1 and navigateToMultiplicity(finite_type)[0] > 1]

In [11]:
getFiniteMultiplicityTypes(all_feature_defs, model_package)

[<7d8b376a-0644-4eef-a60e-2cbd2521abfb «Feature»>, <e2132391-ab40-48ce-8339-f188f4ffca5c «Feature»>, <aec7214d-a245-4da2-a5eb-66d1487afbc7 «Feature»>, <d8815a76-4d96-4b27-80fc-9ec1e513ff2f «Feature»>, <My Seat «Feature»>, <74bcdb7a-1ad9-4124-91af-05e9d02c6bc8 «Feature»>, <Back Wheel «Feature»>, <ddfff54f-3d57-4fcb-bbd4-562dd4140044 «Feature»>, <Front Wheel «Feature»>, <e9e42ab2-6b69-4cf5-81a8-6eba244b1d75 «Feature»>, <driver «Feature»>, <driven «Feature»>, <8d6e35fb-539d-4adf-9a0f-3a651987eda8 «Feature»>, <ce5aaff2-7868-4b89-8fb3-78b4b08a8f60 «Feature»>, <30d92b09-5cdb-4d7f-8c9b-fb3f2b178ee5 «Feature»>, <My Bike «Feature»>]
Trying to find multiplicity on type <7d8b376a-0644-4eef-a60e-2cbd2521abfb «Feature»>
Found non-literal multiplicity range on type <7d8b376a-0644-4eef-a60e-2cbd2521abfb «Feature»>
Trying to find multiplicity on type <e2132391-ab40-48ce-8339-f188f4ffca5c «Feature»>
Found non-literal multiplicity range on type <e2132391-ab40-48ce-8339-f188f4ffca5c «Feature»>
Trying to 

[]

In [12]:
getFiniteMultiplicityTypes(all_class_defs, model_package)

[<A Bike «Classifier»>, <A Seat «Classifier»>, <Wheel «Classifier»>]
Trying to find multiplicity on type <A Bike «Classifier»>
Found literal value on type <A Bike «Classifier»>
Trying to find multiplicity on type <A Bike «Classifier»>
Found literal value on type <A Bike «Classifier»>
Trying to find multiplicity on type <A Seat «Classifier»>
Found literal value on type <A Seat «Classifier»>
Trying to find multiplicity on type <A Seat «Classifier»>
Found literal value on type <A Seat «Classifier»>
Trying to find multiplicity on type <Wheel «Classifier»>
Found non-literal multiplicity range on type <Wheel «Classifier»>


[]

In [13]:
def getSingleFeatMultiClass(features_list, package):
    if printing_level == "TRACE":
        print("Running single feature, multi class check")
    model_types = [each_type for each_type in features_list
                      if each_type.qualifiedName.split('::')[0] == package
                     ]
    single_features = [finite_type for finite_type in model_types if
            len(navigateToMultiplicity(finite_type)) == 1 and navigateToMultiplicity(finite_type)[0] == 1]
    
    if printing_level == "TRACE":
        print("Found features of single multiplicity:")
        print(single_features)
    
    multi_class = [single_feat for single_feat in single_features
                   if len(navigateToMultiplicity(single_feat.type[0])) == 0 or 
                   len(getFiniteMultiplicityTypes(single_feat.type, package)) > 0]
    
    if printing_level == "TRACE":
        print("Found features of single multiplicity with non-single multiplicity types:")
        print(multi_class)
    
    return multi_class

In [14]:
single_features_list = getSingleFeatMultiClass(all_feature_defs, model_package)

Running single feature, multi class check
Trying to find multiplicity on type <7d8b376a-0644-4eef-a60e-2cbd2521abfb «Feature»>
Found non-literal multiplicity range on type <7d8b376a-0644-4eef-a60e-2cbd2521abfb «Feature»>
Trying to find multiplicity on type <e2132391-ab40-48ce-8339-f188f4ffca5c «Feature»>
Found non-literal multiplicity range on type <e2132391-ab40-48ce-8339-f188f4ffca5c «Feature»>
Trying to find multiplicity on type <aec7214d-a245-4da2-a5eb-66d1487afbc7 «Feature»>
Found non-literal multiplicity range on type <aec7214d-a245-4da2-a5eb-66d1487afbc7 «Feature»>
Trying to find multiplicity on type <d8815a76-4d96-4b27-80fc-9ec1e513ff2f «Feature»>
Found non-literal multiplicity range on type <d8815a76-4d96-4b27-80fc-9ec1e513ff2f «Feature»>
Trying to find multiplicity on type <My Seat «Feature»>
Found literal value on type <My Seat «Feature»>
Trying to find multiplicity on type <My Seat «Feature»>
Found literal value on type <My Seat «Feature»>
Trying to find multiplicity on typ

In [15]:
single_features_list

[<Back Wheel «Feature»>, <Front Wheel «Feature»>]

## Defining Unrolling Rule:
### Subset the non-1 multiplicity type and then redefine features to have the new 1-multiplicity types type them

In [16]:
# TODO: Need to develop methods for adding model elements in PyMBE library.